<a href="https://colab.research.google.com/github/safeedhaps963-lab/-dpproject.ipynb-/blob/main/DP1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import libraries
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from  torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.metrics import accuracy_score,classification_report

In [ ]:
#Load and clean dataset
data=pd.read_csv("/content/heart.csv")
data

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0
...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,M,TA,110,264,0,Normal,132,N,1.2,Flat,1
914,68,M,ASY,144,193,1,Normal,141,N,3.4,Flat,1
915,57,M,ASY,130,131,0,Normal,115,Y,1.2,Flat,1
916,57,F,ATA,130,236,0,LVH,174,N,0.0,Flat,1


In [ ]:
#remove missing value
data.dropna(inplace=True)

In [ ]:
#encode categorical columns
label_cols = ['Age','Sex','ChestPainType','RestingBP','Cholesterol']

for col in label_cols:
  data[col]=LabelEncoder().fit_transform(data[col])

In [ ]:
#prepare features and labels
x=data.drop("loan_status",axis=1).values
y=data['loan_status'].values

KeyError: "['loan_status'] not found in axis"

In [ ]:
#normalize features
scaler=StandardScaler()
x=scaler.fit_transform(x)

NameError: name 'x' is not defined

In [ ]:
#split data into train or test
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [ ]:
#convert to pytorch tensors
x_train = torch.tensor(x_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
x_test = torch.tensor(x_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)

In [ ]:
#create DataLoaders
train_ds=TensorDataset(x_train,y_train)
test_ds=TensorDataset(x_test,y_test)
train_loader=DataLoader(train_ds,batch_size=32,shuffle=True)
test_loader=DataLoader(test_ds,batch_size=32)

In [ ]:
#define the neural network
class LoanClassifier(nn.Module):
  def __init__(self):
        super(LoanClassifier,self).__init__()
        self.net =nn.Sequential(
            nn.Linear(13, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, 1),
            nn.Sigmoid()
        )
  def forward(self, x):
        return self.net(x)
model=LoanClassifier()

In [ ]:
#Define loss function and optimizer
criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=0.001)

In [ ]:
#Train the Model
for epoch in range(100):
    model.train()
    total_loss = 0
    for xb, yb in train_loader:
        optimizer.zero_grad()
        outputs = model(xb)
        loss = criterion(outputs, yb)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader):.4f}")


In [ ]:
#Evaluate the model
model.eval()
preds = []
y_true = []
with torch.no_grad():
    for xb, yb in test_loader:
        outputs = model(xb)
        _, predicted = torch.max(outputs, 1)
        preds.extend(predicted.tolist())
        y_true.extend(yb.tolist())

print("\nAccuracy:", accuracy_score(y_true, preds))
print("\nClassification Report:\n", classification_report(y_true, preds))

In [ ]:
#Confusion matrix
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

cm = confusion_matrix(y_true, preds)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Approved", "Rejected"])
disp.plot()
plt.show()

In [ ]:
#Save model weights
torch.save(model.state_dict(), 'loan_classifier_model.pt')
print("Model saved as loan_classifier_model.pt")